In [2]:
import json
import os
import h5py
import numpy as np, numpy.linalg as nlg
import activeSearchInterface as ASI

In [3]:
h5f = h5py.File('/zfsauton/project/public/ben/Xmscocoval2014.h5', 'r')
X = h5f['data']
X = X[:]
h5f.close()

In [4]:
with open('/zfsauton/project/public/ben/mscocoval2014filenames.txt', 'r') as fh:
    filenames = [line.split('/')[-1].rstrip() for line in fh.readlines()]

In [5]:
#load mscoco annotations
data_root = '/zfsauton/project/public/region_of_interest/data'
data = json.load(open(data_root +'/annotations/instances_val2014.json'))
categories = data['categories']
category_lookup = {d[u'id']:d for d in categories}
#create a map from image file name to anotation
id_to_fname = {obj['id']:str(obj['file_name']) for obj in data['images']}

fname_to_annotation = {key:[] for key in id_to_fname.values()}#init dictionary with keys and empty lists
for obj in data['annotations']: #dictionary of annotations. Map image id to filename
    fname_to_annotation[id_to_fname[obj['image_id']]].append(obj)
    
super_cat_lookup = {} #super categories and its cateogory ids that fall under it
for key,obj in category_lookup.items():
    if obj[u'supercategory'] in super_cat_lookup:
        super_cat_lookup[obj[u'supercategory']].append(obj[u'id'])
    else:
        super_cat_lookup[obj[u'supercategory']] = [obj[u'id']]

super_cat_rev_lookup = {} #reverse lookup
for key,obj in category_lookup.items():
    super_cat_rev_lookup[obj[u'id']] = obj[u'supercategory']

In [6]:
# target_sc = 'furniture'
# Y = []
# for fid in filenames:
#     pos = [super_cat_rev_lookup[d['category_id']] == target_sc for d in fname_to_annotation[fid]]
#     Y.append(any(pos))
target_c = 'airplane'
Y = []
for fid in filenames:
    pos = [category_lookup[d['category_id']]['name'] == target_c for d in fname_to_annotation[fid]]
    Y.append(any(pos))
Y = np.array(Y)

In [7]:
Xs = X[:, 4096:]
pi = Y.sum() * 1.0 / Y.shape[0]
sparse = False
verbose = True
alpha = 0.0
prms = ASI.Parameters(pi=pi, sparse=sparse, verbose=verbose, alpha=alpha)

In [8]:
n_init = 1
init_pt = Y.nonzero()[0][np.random.choice(len(Y.nonzero()[0]), n_init, replace=False)]
init_labels = {p:1 for p in init_pt}

In [9]:
kAS = ASI.linearizedAS(prms)

In [10]:
kAS.initialize(Xs.T, init_labels)

Constructing C
Time for constructing C: 6.02053999901
Inverting C
Time for inverse: 3.78080487251
Done with the initialization.


In [11]:
hits = [n_init]
K = 1000
for _ in range(K):
    idx = kAS.getNextMessage()
    print(idx, Y[idx])
    kAS.setLabelCurrent(Y[idx])
    hits.append(hits[-1]+Y[idx])

(39784, False)
Iter: 1, Selected: 39784, Hits: 1, Time: 1.854878
(24742, False)
Iter: 2, Selected: 24742, Hits: 1, Time: 1.760697
(34041, False)
Iter: 3, Selected: 34041, Hits: 1, Time: 1.725915
(32293, False)
Iter: 4, Selected: 32293, Hits: 1, Time: 1.684452
(24751, False)
Iter: 5, Selected: 24751, Hits: 1, Time: 1.732927
(31280, True)
Iter: 6, Selected: 31280, Hits: 2, Time: 1.731969
(27869, True)
Iter: 7, Selected: 27869, Hits: 3, Time: 1.743217
(39534, True)
Iter: 8, Selected: 39534, Hits: 4, Time: 1.761204
(11045, True)
Iter: 9, Selected: 11045, Hits: 5, Time: 1.759176
(8538, True)
Iter: 10, Selected: 8538, Hits: 6, Time: 1.785746
(28196, True)
Iter: 11, Selected: 28196, Hits: 7, Time: 1.856283
(38136, True)
Iter: 12, Selected: 38136, Hits: 8, Time: 1.819193
(31715, True)
Iter: 13, Selected: 31715, Hits: 9, Time: 1.769344
(6445, True)
Iter: 14, Selected: 6445, Hits: 10, Time: 1.751085
(16806, True)
Iter: 15, Selected: 16806, Hits: 11, Time: 1.736971
(26155, True)
Iter: 16, Selecte

In [12]:
normalize = True
wnprms = ASI.WNParameters(normalize=normalize, pi=pi, sparse=sparse, verbose=verbose)
wnAS = ASI.weightedNeighborAS(wnprms)
wnAS.initialize(Xs.T, init_labels)

Done with the initialization.


In [13]:
hits2 = [n_init]
K = 1000
for _ in range(K):
    idx = wnAS.getNextMessage()
    print(idx, Y[idx])
    wnAS.setLabelCurrent(Y[idx])
    hits2.append(hits2[-1]+Y[idx])

(1151, False)
NNAS -- Iter: 1, Selected: 1151, Hits: 1, Time: 0.386185
(31280, True)
NNAS -- Iter: 2, Selected: 31280, Hits: 2, Time: 0.347811
(25296, False)
NNAS -- Iter: 3, Selected: 25296, Hits: 2, Time: 0.346373
(11512, False)
NNAS -- Iter: 4, Selected: 11512, Hits: 2, Time: 0.375028
(29661, True)
NNAS -- Iter: 5, Selected: 29661, Hits: 3, Time: 0.339408
(12811, True)
NNAS -- Iter: 6, Selected: 12811, Hits: 4, Time: 0.344754
(13459, True)
NNAS -- Iter: 7, Selected: 13459, Hits: 5, Time: 0.336276
(29001, False)
NNAS -- Iter: 8, Selected: 29001, Hits: 5, Time: 0.339680
(26133, True)
NNAS -- Iter: 9, Selected: 26133, Hits: 6, Time: 0.344882
(27294, True)
NNAS -- Iter: 10, Selected: 27294, Hits: 7, Time: 0.346448
(15395, True)
NNAS -- Iter: 11, Selected: 15395, Hits: 8, Time: 0.345371
(25249, True)
NNAS -- Iter: 12, Selected: 25249, Hits: 9, Time: 0.355154
(39441, True)
NNAS -- Iter: 13, Selected: 39441, Hits: 10, Time: 0.347054
(19833, True)
NNAS -- Iter: 14, Selected: 19833, Hits: 11

In [ ]:
init_pt